In [339]:
#imporn nessasary packages
import pandas as pd
import numpy as np
import re
#from autocorrect import spell
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
from nltk.stem import LancasterStemmer
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from itertools import combinations

In [340]:
df = pd.read_csv("/Users/FOMIOLNY/desktop/Fedcentral_Assets.csv")
df.head(3)

,ID,Date Uploaded,Date Approved,Display Name,Type of File,Author (name),Author (email),Description,Short Description,File Use,...,Downloadable,Packages,Marketing Materials,Design File,Graphics,Best Practices,Research & reports,Training Materials,Public Tag Names,type
0,https://fedcentral.org/assets/view/375722,10/15/15 20:56,10/15/15 20:56,MetroWest Able Presentation Audio,mp3,Rina Krevat,rina.krevat@jewishfederations.org,Jewish Federation of Greater MetroWest NJMetro...,NaN,full,...,Y,Access Granted Fedovation,N,N,N,N,N,N,disability | 2014 | fedovations | jewish feder...,audio
1,https://fedcentral.org/assets/view/375723,10/15/15 20:56,10/15/15 20:56,RAISE Presentation Audio,mp3,Rina Krevat,rina.krevat@jewishfederations.org,Jewish Federation of Greater OrlandoRAISE (Rec...,NaN,full,...,Y,Access Granted Fedovation,N,N,N,N,N,N,disability | 2014 | jewish federation of great...,audio
2,https://fedcentral.org/assets/view/375801,10/16/15 14:20,10/16/15 14:20,Beyond Happy Hour Full Presentation Audio,mp3,Rina Krevat,rina.krevat@jewishfederations.org,Beyond Happy Hour: Solutions to the Young Adul...,NaN,full,...,Y,Beyond Happy Hour Fedovation,N,N,N,N,N,N,2014 | fedovations | audio,audio


In [341]:
df2 = pd.read_csv("/Users/FOMIOLNY/desktop/post_question.csv")
df2.head(3)

,ID,Type,Title,Answers,Comments,Hi-Fives,Downloads,Score
0,3137423,Question,I have created a group on FedCentral for Commu...,153,0,5,0,903
1,3149073,Question,Have you created a Israel at 70 logo?,22,0,2,0,141
2,3128136,Question,Does your Federation publish a community newsl...,23,0,0,0,140


In [342]:
df['Display Name copy'] = df['Display Name']
df2['Title copy'] = df2['Title']

In [343]:
df['Display Name copy'] = df['Display Name copy'].str.lower()
df2['Title copy'] = df2['Title copy'].str.lower()

In [344]:
df['Display Name copy'] = df['Display Name copy'].str.replace(r'(?!@)[^\w\s]','')
df2['Title copy'] = df2['Title copy'].str.replace(r'(?!@)[^\w\s]','')

In [345]:
#function that replaces NAs with blanks
def replace_na(df,columns):
    df[columns] = df[columns].fillna('')

def apply_tokenization(df, columns):
    stop_words = set(stopwords.words('english'))
    for column in columns:
        df[column] = df[column].apply(word_tokenize)
        #remove stop words    
        for i in range(0,len(df)):
            df[column][i] =[w for w in df[column][i] if w not in stop_words | {''} ]    

            
columns_list = ['Display Name copy']
replace_na(df, columns_list)
apply_tokenization(df, columns_list)

columns_list2 = ['Title copy']
replace_na(df2, columns_list2)
apply_tokenization(df2, columns_list2)

df2.head(3)

/Users/FOMIOLNY/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,ID,Type,Title,Answers,Comments,Hi-Fives,Downloads,Score,Title copy
0,3137423,Question,I have created a group on FedCentral for Commu...,153,0,5,0,903,"[created, group, fedcentral, community, campai..."
1,3149073,Question,Have you created a Israel at 70 logo?,22,0,2,0,141,"[created, israel, 70, logo]"
2,3128136,Question,Does your Federation publish a community newsl...,23,0,0,0,140,"[federation, publish, community, newsletter, c..."


In [346]:
def convert_lemma(df, columns):
    lancaster = LancasterStemmer()

    for column in columns:
        for i in range(0,len(df)):
            df[column][i] =[lancaster.stem(w) for w in df[column][i]]    

convert_lemma(df, columns_list)
df[columns_list].head(3)

convert_lemma(df2, columns_list2)
df2[columns_list2].head(3)

/Users/FOMIOLNY/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Title copy
0,"[cre, group, fedc, commun, campaign, pro, prim..."
1,"[cre, israel, 70, logo]"
2,"[fed, publ, commun, newslet, cal, launch, mont..."


In [ ]:
for index, qp in df2.iterrows():
    print(qp['Title'])    
    print('')
    print('assets:')
    for index,asset in df.iterrows():
        if set(asset['Display Name copy']).issubset(set(qp['Title copy']))==True:
            print('        '+ str(asset['Display Name']))
    print(' ')
    print('===============================================================')

I have created a group on FedCentral for Community Campaign Pros - primarily focusing on strategies for under $1000 (direct mail, telegiving, ephilanthropy etc.) Let me know if you want to join!

assets:
 
Have you created a Israel at 70 logo?

assets:
 
Does your Federation publish a community newsletter? If so, what is it called? We are launching a monthly e-newsletter with news from our partners and are searching for a name that's Jewish but not too Jewish.

assets:
 
Is this a draconian contract from our Graphic Designer?

assets:
 
Teen Israel Scholarship $
Does your federation provide any $ towards teen Israel travel? What relationship, if any, does the program have to local youth groups and/or synagogues? Please share URL of your program's online presence in answer, thanks!

assets:
 
Can you please share your 2017 Film Festival films?

assets:
 
Gifts at Women's Philanthropy LOJ?

assets:
 
How is your Luminate transition going?

assets:
 
Does anyone have donor advised funds w

 
Hi, I'm looking for good direct mail samples for Annual Campaign for use at end-of-Campaign or adaptable for that purpose. Thanks!

assets:
 
Major Women's Event Speaker: Who was a draw for attendance? What case for giving speakers have made the largest impact on campaign? 

assets:
 
I'm looking to give women who receive a Lion or Pom pin a card to accompany it--one that shares more about its background and encourages them to wear it often and proudly. Does anyone have an example of text that they use for something like this?

assets:
 
We are doing a video of individual recipient impact stories (not agencies) Anyone have examples of these types of videos or scripts for them? We will be getting recipients of PJ, Young Adult, JCRC security, Holocaust survivors, camp, Israel Experience.

assets:
 
We are considering getting a handicap accessible van for our small community, has anyone else done this?

assets:
 
Do any Federations send thank you letters when donors pay off their campai

 
Chicago's YLD Big Event Fundraiser marketing materials

assets:
        Marketing for Fundraising Event
 
The JCCA is currently looking into hosting a one or two-day Facility Manager/Director's Conference. I know some of us own/operate campuses.  Would this be something that Federations would be interested in collaborating on?

assets:
 
What is your Federation's kashrut policy?

assets:
 
Looking for a professional Israeli dance troupe, other than Bat Sheva, to bring here for Israel @ 70 next year. Any suggestions?

assets:
 
to those of you who experienced bomb threats today, are you seeing an unsual media interest this time.  The first time this happened, we had no media inquiries.  Today, we have been called by all four network television channels, NPR radio, and others.

assets:
 
Does the Federation staff in your community get a professional courtesy discount with the JCC?

Thank you!
Sheila

assets:
 
[Updated Resources] Federations Working to Promote Religious Pluralism in Is

 
Recommendations for freelance graphic designers

assets:
 
I was recently hired as a Director of Impact Assessment & Learning – basically an in-house evaluation and impact management role. Do any of your federations have a similar position? Would love to connect with any counterparts!

assets:
 
Interactive Rosh Hashanah App from Toronto

assets:
 
We are looking to bring in Jane Weitzman for our Lion Pom event.  Has anyone had her in for a campaign event?  If so, what was the feedback?  Is her message more about philanthropy in general or does she have a specific message and tie to Federation?

assets:
 
Who is the signer of your acknowledgement letters? Lay leader, CEO, development staff? Thanks! 

Also, can you share any new language you are using for 2018? Our letter needs a rewrite!

assets:
        LEADERS
 
Does anyone have an application for security grants? We are considering funding security grants to our community. Can you share the grant guidelines, process and grant leve

 
Good morning, 
We are looking at some new software to replace a legacy system. I would like to know what donor management system you are using on the front-end and what accounting system is being used on the back-end. 
Thanks
Les

assets:
 
Has anyone had Erica Brown speak at a big event?

assets:
 
What is the breakout of how you advertise/promote your organization? Direct mail, Facebook/social media, Print (what type of publications), other? And how are you targeting prospective donors?

assets:
 
Indiana University: Lilly Family School of Philanthropy Resources

assets:
 
Does anyone use a renewal process for donors under $100? ie, thanks for your past gift, if we don't hear from you we will renew you at the same level as last year?

assets:
 
How does your community honor Jewish Educators? Do you have a community wide celebration or award program? Does each individual school have their own recognition program?

assets:
 
Can anyone recommend a speaker for a Planned giving day?  O

...

        Ida
 
Marketing a Community Mission

assets:
 
Does anybody have any  information they use for the Pomegranate Society. I would love to hear from you. Broward County has just introduced the level.

assets:
        The Pomegranate
 
Can anyone recommend grants management software?

assets:
 
Does anyone have a good system for gathering metrics from endowment development officers?

assets:
 
Event Solicitation Strategies

assets:
 
Has anyone used Chuck Todd, Moderator of Meet The Press, as a speaker?

assets:
 
Please post your most successful Passover Appeal letter (from past years) and let us know how it performed.

assets:
 
Cardozo Society-topics/speakers/events. Looking for recommendations and suggestions. Please share your ideas.

assets:
 
We are planning an exclusive and elite Young Leadership Mission to Israel.  Do you have any itineraries for a similar trip that you have run to Israel that you could share? We are looking for unique and innovative activities for our gro

 
Can you recommend a motivational speaker for a Women's Philanthropy retreat?

assets:
 
We have two families in our community looking to join a bar/bat mitzvah mission this year. Does anyone have something that they could join? Please let me know, julia@jewishnevada.org. Thanks!

assets:
 
Outgoing Board Member Gifts

assets:
 
Social Media Policy

assets:
 
Young Adult Division Summer Kick Off

assets:
 
How does your community refer to solicitors? (i.e. canvassers, relationship managers, ambassadors, etc.)

assets:
 
Takeaway ideas for annual Lion dinner

assets:
 
Does your Federation recognize past campaign chairs on a plaque or in some other way in your building? We would appreciate hearing how you do this in your community. Thanks!

assets:
 
Question for publication

assets:
 
How does your Federation promote its financial aid for Israel travel programs? Have you partnered with any Israel travel programs to produce content?

assets:
 
Does you support J Street U?
We have had a

 
Are there any Federations organizing a Family Mission to Israel in spring 2018 or summer 2018? (and are open to accepting a family with three kids from Indianapolis)

assets:
 
Pledge Card Redesign

assets:
 
New Senior Services Group

assets:
 
Has anyone tried any sort of men's giving circle? Or some other type of program to engage men in giving (both of time and resources)?

assets:
 
We'd like to implement Google Tag Manager on our site. Has anyone done this and can I simply follow the Google online process to set this up?

assets:
 
I was invited to do two NPR interviews , one in MI and one in NYC.  Here are the links.
https://cpa.ds.npr.org/michigan/audio/2017/02/20170228_SS_Full.mp3
http://www.wnyc.org/story/bomb-threats-ripple-across-american-jewish-communities

assets:
 
New Website Rollout Plan

assets:
 
Falling short of campaign goal

assets:
 
Who is producing their own campaign video this year?

assets:
 
Need a TED Talks style speaker with name recognition for $10-20K.

 
Has anyone set up (in-house) a "dark site" and have insight? 
(A dark web site is a pre-made web site activated when emergency occurs, with salient info written in advance and specifics added before activating it.)

assets:
 
Israel decorations/swag?

assets:
 
Descriptions of board/committee positions

assets:
 
Yesterday's article by Yehuda Kurtzer in eJewishphilanthropy is a worthy read and I thought it would good to see what people thought of it.

assets:
 
We are looking for development director and executive director job descriptions for a small to medium size federation. Any assistance would be appreciated. 

assets:
 
Developing Successful Corporate Sponsorship Programs

assets:
        Developing Successful Corp. Sponsorship Programs
        Develop Successful Corporate Sponsorship Programs
 
Constituent Profile Form

assets:
 
I'm looking for pros and cons of Directed Giving in prep for a staff discussion. Want to weigh in?

assets:
 
Does anyone have a protocol for evaluat

 
I'm looking for a speaker or an out of the box event idea for a Women's Philanthropy Spring Closing event.  We have a speaker budget of about $2,500.  We have already had Siggy Flicker and Sima Goel, I see them suggested a lot.  Any new ideas?  Thanks!

assets:
        Women's Philanthropy Event Ideas
 
Does your community have separate Federation/Foundation allocations and grant making, but a process in place where community needs and priorities are jointly discussed?

assets:
 
How does your Cardozo Society (legal professionals group) run?

assets:
 
Speaker Recomendation

assets:
 
Can anyone share sample campaign ads that they have placed in a newspaper/magazine? Any size is fine. Thank you!

assets:
 
Can anyone share their investment policy guidelines?

assets:
 
Please share your Federation's "Statement of Understanding" document for partner agencies who receive core or programmatic funding.

assets:
 
Has anyone ran or considered a national/competitive campaign like the one p